In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
import zipfile

with zipfile.ZipFile("./chest-xray-pneumonia.zip", 'r') as z:
    
    z.extractall()

In [ ]:
from ast import increment_lineno
import os
import shutil
import random
import numpy as np
import pandas as pd
import cv2
import skimage
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [ ]:
lables=["PNEUMONIA","NORMAL"]
image_size=128

In [ ]:
def get_data(data_dir):
  data=[]
  for label in lables:
    path=os.path.join(data_dir,label)
    class_num=lables.index(label)
    for img in os.listdir(path):
      try:
        img_arr=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        resized_arr=cv2.resize(img_arr,(image_size,image_size))
        data.append(resized_arr)
      except Exception as e:
        print(e)
  return np.array(data)


In [ ]:
train=get_data('./chest_xray/chest_xray/train')
test=get_data('./chest_xray/chest_xray/test')
val=get_data('./chest_xray/chest_xray/val')

In [ ]:
pneumonia_dir='./chest_xray/train/PNEUMONIA'
pneumonia=os.listdir("./chest_xray/train/PNEUMONIA")

#Data Agumnetation



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense,Flatten,Input,MaxPooling2D,Conv2D,Dropout
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=0.4,
    vertical_flip=0.4,
    rotation_range=40,
    shear_range=0.2,
    width_shift_range=0.4,
    height_shift_range=0.4,
    fill_mode="nearest"
)
vaild_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator=train_datagen.flow_from_directory(
    './chest_xray/chest_xray/train',
    batch_size=32,
    target_size=(128,128),
    class_mode='categorical',
    shuffle=True,
    seed=42,
    color_mode='rgb'
)


valid_generator=vaild_datagen.flow_from_directory(
    './chest_xray/chest_xray/val',
    batch_size=32,
    target_size=(128,128),
    class_mode='categorical',
    shuffle=True,
    seed=42,
    color_mode='rgb'
)


test_generator=test_datagen.flow_from_directory(
    './chest_xray/chest_xray/test',
    batch_size=32,
    target_size=(128,128),
    class_mode='categorical',
    shuffle=True,
    seed=42,
    color_mode='rgb'
)

In [ ]:
class_lables=train_generator.class_indices
class_name={value:key for key,value in class_lables.items()}

In [ ]:
base_model=VGG19(input_shape=(128,128,3),include_top=False,weights="imagenet")
for layer in base_model.layers:
    layer.trainable=False
x=base_model.output
flat=Flatten()(x)
class_1=Dense(4608,activation='relu')(flat)
dropout=Dropout(0.2)(class_1)
class_2=Dense(1152,activation="relu")(dropout)
output=Dense(2,activation="softmax")(class_2)
model_01=Model(base_model.input,output)

model_01.summary()

In [ ]:
model_01.evaluate(test_generator)

In [ ]:

filepath = "model.h5" 

#
# es = EarlyStopping(monitor="val_loss", verbose=1, mode="min", patience=4)
# cp = ModelCheckpoint(filepath, monitor="val_loss", save_best_only=True, save_weights_only=False, verbose=1)
# lrr = ReduceLROnPlateau(monitor="val_accuracy", patience=3, verbose=1, factor=0.5, min_lr=0.00001)


model_01.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy"])

history = model_01.fit(train_generator,
                        steps_per_epoch=50,
                        epochs=2,
                        validation_data=valid_generator)

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image


def prepare_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)  
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0 
    return img_array


img_path = '/content/chest_xray/test/PNEUMONIA/person100_bacteria_477.jpeg'  
target_size = (128, 128) 

input_image = prepare_image(img_path, target_size)


predictions = model_01.predict(input_image)

predicted_class = np.argmax(predictions, axis=1) 
if predicted_class==1:
  print("PENUMIOINA")
elif predicted_class==0:
  print("Nromal")
print("Predicted class index:", predicted_class)
